In [0]:
%run ./_udf_utils

In [0]:
#%pip install python-dotenv

In [0]:
import os
from dotenv import load_dotenv
import json

# load environment variables from .env
load_dotenv() 
## call API_KEY saved in .env
api_key = os.getenv("FOREIGNER_DATA_API_KEY") 
base_url = "https://api.odcloud.kr/api"

# get a list of endpoint from a textfile
with open("resident_shortterm_endpoints.txt", "r") as f:
    endpoints_to_read = [line.strip() for line in f]

total_results = []
i = 1
# loop through each endpoint to create a list of tuple  
for ep in endpoints_to_read:
    print(f"processing.. {i}/{len(endpoints_to_read)}")
    result_data = read_data(base_url, ep, api_key)
    total_results.append((ep, json.dumps(result_data, ensure_ascii=False))) # ensure_ascii: keep korean format
    i += 1

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType

spark = SparkSession.builder.appName("resident_shortterm_ingest").getOrCreate()
shortterm_data = total_results
shortterm_schema = StructType([
    StructField('endpoint', StringType(), False),
    StructField('data', StringType(), True)
])

shortterm_df = spark.createDataFrame(data = shortterm_data, schema = shortterm_schema)

In [0]:
shortterm_df.write.format("delta").mode("overwrite").saveAsTable("workspace.growth_poc.bronze_residents_shortterm")